# Практический кейс 2

## Анализ данных магазина розничной торговли. Построение простой рекомендательной системы

При построении простой рекомендательной системы, ответим на вопросы:
- какие пары товаров покупались вместе чаще всего?
- какие товары чаще всего покупались вместе с данным товаром?

Торговая сеть предоставила вам данные о покупках своих клиентов, представляющие собою таблицу со следующими столбцами:
- `InvoiceNo` - номер чека. Товары с одинаковым `InvoiceNo` были приобретены одним покупателем в одной покупке.
- `StockCode` - универсальный идентификатор товара в базе данных магазина (один и тот же товар имеет единый `StockCode` во всех чеках)
- `Description` - название товара
- `Quantity` - количество товаров данного типа в чеке
- `UnitPrice` - цена одной единицы товара
- `InvoiceDate` - дата совершения покупки
- `CustomerID` - идентификационный номер пользователя (если покупка совершалась с каким-нибудь идентификатором, например с помощью скидочной карты магазина). Много покупок проходит без идентификации пользователя, поэтому в этой колонке может быть много пропусков.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("online retail.csv")

Saving online retail.csv to online retail.csv


In [3]:
df.shape

(100000, 7)

In [ ]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0


In [4]:
df_first_part = df[['InvoiceNo', 'StockCode', 'Description']]

Нам будет удобнее работать с данными, если `id` товаров будут идти от 1 до числа уникальных товаров в датафрейме. 

Поэтому создадим новые удобные id товаров
(создадим словарь новых индексов, в котором пронумеруем товары числами от 1 до количества уникальных товаров в этом датафрейме и присвоим товарам данные id)

In [5]:
unique_sorted_stock_codes = sorted(df_first_part["StockCode"].unique())

In [6]:
len_StockCode_list = len(unique_sorted_stock_codes)
len_StockCode_list


3128

In [7]:
id_codes = np.arange(0,len_StockCode_list)
id_codes 

# проверка, что списки одинаковые
print("Длина списков одинакова", len(unique_sorted_stock_codes) == len(id_codes))

#создание словаря
new_codes = dict(zip(unique_sorted_stock_codes,id_codes))

Длина списков одинакова True


При построении простой рекомендательной системы, ответим на вопросы:
- какие пары товаров покупались вместе чаще всего?
- какие товары чаще всего покупались вместе с данным товаром?


Для этого построим "матрицу смежности" ( число на пересечении `i`-й строки и `j`-го столбца отражают в скольки чеках были одновременно два товара с индексами `i` и `j`)

Для этого:

0) cоздать матрицу из нулей;
1) сгруппировать объекты по чекам;
внутри каждой группы:
2) сгенерировать попарные комбинации товаров внутри одного чека (с помощью Intertools);
для каждой сгенерированной пары:
3) вычислить индексы i и j каждого из товаров в паре
4) прибавить 1 к ячейке матрицы с индексами [i,j]

Примечание: если два товара встречались комбинациях по 2 в разном порядке (напр. (item_1, item_2) и (item_2, item_1)), то единичку ставим в ячейку, у которой номер строки меньше (если i < j, то +1 к элементу [i,j], иначе +1 к элементу [j,i]). 



In [8]:
import itertools

In [10]:
# Создадим нулевую матрицу
item_pairs_counts = np.zeros((len(new_codes), len(new_codes)))
# 1) сгруппировать объекты по чекам;
for invoice_number, invoice_group in df_first_part.groupby(['InvoiceNo']):
# внутри каждой группы:
# 2) сгенерировать попарные комбинации товаров внутри одного чека;
# для каждой сгенерированной пары:
# 3) вычислить индексы i и j каждого из товаров в паре
    for item_1, item_2 in itertools.combinations(list(invoice_group["StockCode"].unique()), 2):   # можно вместо unique, set использовать
#         print(item_1,item_2)
        i = new_codes[item_1]
        j = new_codes[item_2]
# 4) прибавить 1 к ячейке матрицы с индексами [i,j] и сделать верхоугольную матрицу
        if i < j:
            item_pairs_counts[i,j] += 1
        else:
            item_pairs_counts[j,i] += 1
       
item_pairs_counts      

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#Проверка матрицы.

Проверка, в скольких чеках встречалась пара товаров со `StockCode=85123A` и со `StockCode=71053`:

In [11]:
counter = 0
for invoice_num, invoice_group in df_first_part.groupby("InvoiceNo"):
    s = set(invoice_group['StockCode'])
    if '71053' in s and '85123A' in s:
        counter += 1
print(counter)


36


In [12]:
# Другой способ проверки - на основе основного кода. 
count = 0
for group_key, df in df_first_part.groupby(["InvoiceNo"]):
    for item_12 in itertools.combinations(set(df["StockCode"]), 2):
        if item_12 == ("85123A", "71053") or item_12 == ("71053", "85123A"):
            count += 1
# проверила, что может быть в одном инвойсе несколько строк с одинаковым товаром
    # print(len((df["StockCode"])) != len((df["StockCode"]).unique()))
count

36

Таким образом пара товаров со `StockCode=85123A` и со `StockCode=71053` встречалась одновременно в 36 чеках.

Теперь получим это же число чеков, в которых эта пара товаров встречалась вместе с помощью вычисленной матрицы.

In [ ]:
item_pairs_counts[new_codes['71053'], new_codes['85123A']]
# изначально тест у меня был с ошибкой. Как выяснилось из-за того, что в инвойсе могло быть две строки с одинаковым товаром
# проверила, что такие случаи имеют место быть.В результате добавила в основном коде уникальные значения товаров в каждом инвойсе 


36.0

Проверка для любого другого товара с другими `StockCode`

(товары определяются или вводом значений, или случайным образом (если товары не определены)

In [18]:
if input("У вас есть искомые товары (ответ да/нет)?") == "да":
    a = input("Введите StockCode первого товара")
    b = input("Введите StockCode второго товара")
else:
# Определение случайным образом StockCode товаров a b
    StockCode_unique = np.array(sorted(df_first_part['StockCode'].unique()))
    a = np.random.choice(StockCode_unique, size=1, replace=True, p=None)[0]
    b = np.random.choice(StockCode_unique, size=1, replace=True, p=None)[0]

# проверка по изначальной таблице
counter = 0
for invoice_num, invoice_group in df_first_part.groupby("InvoiceNo"):
    s = set(invoice_group['StockCode'])
    if a in s and b in s:
        counter += 1

# проверка из матрицы
print("Код первого товара:",a)
print("Код второго товара:",b)
print("Кол-во пересечений в таблице:",counter)
print("Кол-во пересечений в матрице:",item_pairs_counts[new_codes[b], new_codes[a]], item_pairs_counts[new_codes[a], new_codes[b]])
# печать выводов
if counter==0:
    print("Пересечений нет")
elif counter == item_pairs_counts[new_codes[b], new_codes[a]] or counter == item_pairs_counts[new_codes[a], new_codes[b]]:
    print("значение числа чеков, в которых есть оба этих товара, определены верны")
else:
    print("значение числа чеков, в которых есть оба этих товара, определены неверно")

У вас есть искомые товары (ответ да/нет)?да
Введите StockCode первого товара85123A
Введите StockCode второго товара71053
Код первого товара: 85123A
Код второго товара: 71053
Кол-во пересечений в таблице: 36
Кол-во пересечений в матрице: 36.0 0.0
значение числа чеков, в которых есть оба этих товара, определены верны


# Определим,какие товары чаще всего покупались вместе с данным товаром (товаром с данным id)

Для оптимизации транспонируем нашу верхнеугольную матрицу, чтобы для пары товаров данные были одинаковые, независимо, по строкам или столбцам мы ищем товар.

In [19]:
item_pairs_counts += item_pairs_counts.T


In [20]:
# проверяем, что матрица стала симметричной
np.allclose(item_pairs_counts, item_pairs_counts.T)

True

Какая пара товаров была куплена вместе наибольшее число раз? 


In [21]:
#подготовка
#для конвертации индекса в Stockcode нам нужно "транспонировать" словарь
new_codes_changed_key_value = {}
for k, v in new_codes.items():
    new_codes_changed_key_value[v] = k

# Нужно определить максимальное значение в матрице c помощью np.argmax (находим индекс максимального значения. в матрице могут быть несколько пар товаров\
# с одинаковым максимальным число сочетаний. np.argmax вернет первое из них)
# Затем определяем StockCode и название товара
# Выводим информацию на печать
max_Index_pairs_counts = np.unravel_index(item_pairs_counts.argmax(), (3128,3128))
print("Максимальное количество комплиментарных товаров:",item_pairs_counts[max_Index_pairs_counts])
print("Товары с наибольшим пересечением:",max_Index_pairs_counts)
print("\n")
print("Товар 1 имеет индекс {}".format(max_Index_pairs_counts[0]),\
          "Товар 1 имеет StockCode  {}:".format(new_codes_changed_key_value[max_Index_pairs_counts[0]]),\
          "Товар 1 из имеет название {}:".format(df_first_part[df_first_part["StockCode"]==\
                                                                           new_codes_changed_key_value[max_Index_pairs_counts[0]]]["Description"].iloc[1]),sep="\n")
print(f"\n")
print("Товар 2 имеет индекс {}".format(max_Index_pairs_counts[1]),\
          "Товар 2 имеет StockCode {}:".format(new_codes_changed_key_value[max_Index_pairs_counts[1]]),\
          "Товар 2 имеет название {}:".format(df_first_part[df_first_part["StockCode"]==\
                                                                           new_codes_changed_key_value[max_Index_pairs_counts[1]]]["Description"].iloc[1]),sep="\n")

Максимальное количество комплиментарных товаров: 189.0
Товары с наибольшим пересечением: (1323, 1324)


Товар 1 имеет индекс 1323
Товар 1 имеет StockCode  22469:
Товар 1 из имеет название HEART OF WICKER SMALL:


Товар 2 имеет индекс 1324
Товар 2 имеет StockCode 22470:
Товар 2 имеет название HEART OF WICKER LARGE:


#Определим, что чаще всего покупали вместе с `KNITTED UNION FLAG HOT WATER BOTTLE` (`stock_code=84029G`)?
Если совпадений одинаковое количество, выведем первый товар






In [63]:
def find_compliment_tovar(index_for_pair, item_pairs_counts):
        max_item_pairs_counts = item_pairs_counts[:,index_for_pair].max()
        index_goods_max = np.where(item_pairs_counts[:,index_for_pair] == max_item_pairs_counts)
        index_goods_max = np.array(index_goods_max)[0][0]
        return index_goods_max, max_item_pairs_counts 


In [66]:
code = df_first_part[df_first_part["Description"] == "KNITTED UNION FLAG HOT WATER BOTTLE"]["StockCode"].iloc[1]
index_for_pair = new_codes[code]
compliment_tovar_index,max_item_pairs_counts  = find_compliment_tovar(index_for_pair, item_pairs_counts)
print("товар имеет комплиментарный товар с индексом: {}".format(compliment_tovar_index))
compliment_tovar_stockcode = new_codes_changed_key_value[compliment_tovar_index]
print("товар имеет комплиментарный товар с StockCode: {}".format(compliment_tovar_stockcode)) 
compliment_tovar_description = df_first_part[df_first_part["StockCode"] == compliment_tovar_stockcode]["Description"].iloc[1]
print("товар имеет комплиментарный товар с названием: {}".format(compliment_tovar_description))
print("количество совпадений с комплиментарным товаром: {}".format(max_item_pairs_counts))


товар имеет комплиментарный товар с индексом: 2215
товар имеет комплиментарный товар с StockCode: 84029E
товар имеет комплиментарный товар с названием: RED WOOLLY HOTTIE WHITE HEART.
количество совпадений с комплиментарным товаром: 82.0


# Определим по входному списку товаров список такой же длины, состоящий из товаров, которые наиболее часто покупаются вместе с каждым товаром из входного списка.



In [67]:
input_items_list = ['84406B', '22585', '20749']

In [71]:
input_items_list = ['84406B', '22585', '20749']
spisok_for_recommend = []
input_items_index_list = [new_codes[s] for s in input_items_list]  
for i in input_items_index_list:
    spisok_for_recommend.append(find_compliment_tovar(i, item_pairs_counts)[0])
spisok_for_recommend    

[2679, 1280, 164]